In [ ]:
%%file cost_func.py

import numpy as np

##cost function
def cost(c,data):
    return np.sum([min(np.sum((c-d)**2,axis=1)) for d in data])


In [ ]:
%%file smpl_prb_func.py

import numpy as np
from cost_func import cost

def smpl_prb(c,data,l):
    phi_temp = cost(c,data)
    return np.array([(min(np.sum((c-d)**2,axis=1)))*l/phi_temp for d in data])

In [ ]:
%%file KmeansParallel_func.py

from __future__ import division
import numpy as np
import sys
import sklearn.cluster

#Kmeans||
#l is oversampling factor

def KmeansParallel(n_clusters, data, l):
    if n_clusters <= 0 or not(isinstance(n_clusters,int)):
        sys.exit("n_cluster is not positive integer")
    
    if l <= 0: 
        sys.exit("l is not positive")
    
    if len(data) < n_clusters: 
        sys.exit("number of data is less than n_clusters")
    
    ##cost function
    # Version 1 - 2 list comprehension, no broadcasting
    # def cost(c,data):
    #     return np.sum([min([euclidean(d,s) for s in c])**2 for d in data])

    # Version 2 - 1 list comprehension and one broadcasting
    def cost(c,data):
        return np.sum([min(np.sum((c-d)**2,axis=1)) for d in data])


    #sampling probability function
    # Version 1 - 2 list comprehension, no broadcasting
    # def smpl_prb(c,data,l):
    #     phi_temp = cost(c,data)
    #     return np.array([(min([euclidean(d,s) for s in c])**2)*l/phi_temp for d in data])

    # Version 2 - 1 list comprehension, 1 broadcasting
    def smpl_prb(c,data,l):
        phi_temp = cost(c,data)
        return np.array([(min(np.sum((c-d)**2,axis=1)))*l/phi_temp for d in data])
    
    num = len(data)
    
    #Step 1 - uniformly sample one point
    c = np.array(data[np.random.choice(range(num),1),])
    
    #Step 2 - cost
    phi = cost(c,data)
    
    #Step 3~6 - get potential centers
    for i in range(np.ceil(np.log(phi)).astype(int)):
        c_add = data[smpl_prb(c,data,l)>np.random.uniform(size = num),]
        c = np.concatenate((c,c_add))
        
    #Step 7
    # Find the closet point in c for each point in data
    # Version 1 - 2 list comprehension, no broadcasting
    # min_c = [np.argmin([euclidean(d,s) for s in c]) for d in data];
    # Version 2 - 1 list comprehension, 1 broadcasting
    min_c = [np.argmin(np.sum((c-d)**2,axis=1)) for d in data];

    ##number of points which is closest to each s in c
    num_closest = [min_c.count(i) for i in range(len(c))]
    
    ##weight
    weight = num_closest/np.sum(num_closest)
    
    #Step 8 - recluster by kmeans++ initialization
    c_final = data[np.random.choice(range(len(c)),size=1,p=weight),]
    data_final = c
    for i in range(n_clusters-1):
        new_prb = smpl_prb(c_final,data_final,l) * weight
        c_fin_add = data[np.random.choice(range(len(c)),size=1,p=new_prb/np.sum(new_prb)),]
        c_final = np.concatenate((c_final,c_fin_add))
    
    #Do k-means with initial centers
    import sklearn.cluster
    km2 = sklearn.cluster.KMeans(n_clusters=n_clusters, n_init=1, init=c_final, max_iter=500, tol=0.0001)
    km2.fit(data);
    
    #return a KMeans type result - including: cluster_centers_, labels_, inertia_
    return km2
    

In [ ]:
%%file test_cost.py

import numpy as np
from numpy.testing import assert_almost_equal
from cost_func import cost

def test_non_negativity():
    for i in range(10):
        data = np.random.normal(size=(10,2))
        c = data[np.random.choice(range(10),1),]
        assert cost(c, data) >= 0

def test_full_data_zero():
    for i in range(10):
        data = np.random.normal(size=(10,2))
        c = data
        assert cost(c, data) == 0

def test_c_more_cost_less():
     for i in range(10):
        data = np.random.normal(size=(10,2))
        c_more = data[np.random.choice(range(10),4,replace=False),]
        c = c_more[:2,]
        assert cost(c_more, data) <= cost(c, data)


In [ ]:
%%file test_smpl_prb.py

import numpy as np
from numpy.testing import assert_almost_equal
from cost_func import cost
from smpl_prb_func import smpl_prb

def test_non_negativity():
    l = 3
    for i in range(10):
        data = np.random.normal(size=(10,2))
        c = data[np.random.choice(range(10),1),]
        assert np.alltrue(smpl_prb(c,data,l) >= 0)
        

def test_sum_to_l():
    for i in range(10):
        l = i + 1
        data = np.random.normal(size=(10,2))
        c = data[np.random.choice(range(10),1),]
        assert (np.sum(smpl_prb(c,data,l)) - l) <= 1e-6

def test_in_c_zero():
    l = 2
    for i in range(10):
        data = np.random.normal(size=(10,2))
        choice = np.random.choice(range(10),1)
        c = data[choice,]
        prb = smpl_prb(c,data,l)
        assert np.alltrue(prb[choice,] == 0)



In [ ]:
%%file test_KmeansParallel.py

import numpy as np
import sys
from numpy.testing import assert_almost_equal
from numpy.testing import assert_raises
from KmeansParallel_func import KmeansParallel

def test_level_label():
    for i in range(10):
        data = np.random.normal(size=(10,2))
        k = 3
        assert len(set(KmeansParallel(n_clusters = k, data = data, l = 2*k).labels_)) == k

def test_num_label():
    for i in range(10):
        data = np.random.normal(size=(10,2))
        k = 3
        assert len(KmeansParallel(n_clusters = k, data = data, l = 2*k).labels_) == len(data)

        

In [ ]:
! py.test